In [2]:
from common_imports import *

In [4]:
"""
JSON_Engine is a wrapper over LlamaIndex JSON Query Engine
It takes a JSON prompt and a pydanctic class name as input in the constructor
It uses OpenAI API to generate the output

run method processes the prompt and returns the output in the form of pydantic class object
"""
class JSON_Engine(BaseTool):
    def __init__(self, prompt, class_name, llm_model_name: str = "mistralai/Mixtral-8x7B-Instruct-v0.1",temperature=0.1, api_key_name = "TOGETHER_API_KEY",parse=True,gemma_mode=False):
        self.output_parser = PydanticOutputParser(class_name)
        # self.llm = TogetherLLM(model=llm_model_name, api_key=os.environ[api_key_name], temperature=temperature)
        self.llm = OpenAI(model="gpt-3.5-turbo", temperature=temperature)
        self.json_prompt_str = prompt
        self.class_name = class_name
        self.json_prompt_str = self.output_parser.format(self.json_prompt_str)
        self.json_prompt_tmpl = PromptTemplate(self.json_prompt_str)
        if parse:
            self.p = QueryPipeline(chain=[self.json_prompt_tmpl, self.llm, self.output_parser], verbose=False)
        else:
            self.p = QueryPipeline(chain=[self.json_prompt_tmpl, self.llm], verbose=False)
        self.gemma_mode = gemma_mode
    
    def run(self, **kwargs):
        response = self.p.run(**kwargs)
        if self.gemma_mode:
            pattern = r'{.*}'
            matches = re.findall(pattern, response.message.content, re.DOTALL)
            largest_match = max(matches, key=len, default=None)
            print(largest_match)
            response = self.output_parser.parse(largest_match)
        return response

In [35]:
class TicketSummary(BaseModel):
    summary:str = Field(...,description=
            """
            Summary of all technical details to be included in the ticket.
            """)
    
    priority:str = Field(...,description=
            """
            Priority of the ticket. 
            High - If the issue is critical and needs immediate attention.
            Medium - If the issue is important but can be resolved within a day.
            Low - If the issue is minor and can be resolved within a week.
            """)

class TicketSummaryEngine:
    def __init__(self):
        self.engine = JSON_Engine("""
        Given some issues raised by the user (between <<< and >>>), 
        <<<
        {issues}
        >>>
        
        Summary of the technical details to be included in the ticket:
        """,class_name = TicketSummary,temperature=0.1)
    
    def __call__(self,**kwargs):
        return self.engine.run(**kwargs)

In [75]:
"""
Ticket Json-
tickcet_id
timestamp
review_json_list
content
issue_category
priority
team_assigned
no_of_users_reported
resolved_status
resolution_timestamp
solution
"""

def aggregate_reviews_teamwise(team_assigned_path="datas/team_assigned_result_reviews_relv_Google_Pay_Secure_UPI_payment.json"):
    ticket_summary_engine = TicketSummaryEngine()
    priority_map = {"High":3,"Medium":2,"Low":1}
    with open(team_assigned_path) as f:
        reviews = json.load(f)
    
    team_wise_reviews = {}
    for review in reviews:
        teams = review['metadata']['assigned_teams']
        for team in teams:
            if team not in team_wise_reviews:
                team_wise_reviews[team] = []
            team_wise_reviews[team].append(review)

    # sort reviews by date
    for team in team_wise_reviews:
        team_wise_reviews[team] = sorted(team_wise_reviews[team], key=lambda x: x['metadata']['date_time'], reverse=True)

    
    # Group reviews taking 4 reviews at a time
    for team in team_wise_reviews:
        team_wise_reviews[team] = [team_wise_reviews[team][i:i+4] for i in range(0, len(team_wise_reviews[team]), 4)]
    
    # Team wise tickets
    team_wise_tickets = {}
    for team in team_wise_reviews:
        team_wise_tickets[team] = []
        for reviews in team_wise_reviews[team]:
            ticket = {}
            ticket['ticket_id'] = str(random.randint(100000,999999))
            ticket['timestamp'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            # remove reviews['metadata']['assigned_teams'] and 'reasons'
            reviews = [{'metadata':{k:v for k,v in review['metadata'].items() if k not in ['assigned_teams','reasons']}} for review in reviews]
            ticket['review_json_list'] = reviews
            concat_reviews = "\n".join([", ".join(review['metadata']['issues']) for review in reviews])
            ticket_engine_output = ticket_summary_engine(issues=concat_reviews)
            priority = ticket_engine_output.priority
            ticket['content'] = ticket_engine_output.summary
            ticket['content'] = concat_reviews
            # ticket['issue_category'] = list(set([issue.split(' Team')[0] for review in reviews for issue in review['metadata']['assigned_teams']]))
            ticket['priority'] = priority_map[priority]
            ticket['team_assigned'] = team
            ticket['no_of_users_reported'] = len(reviews)
            ticket['resolved_status'] = 0
            ticket['resolution_timestamp'] = ""
            ticket['solution'] = ""
            team_wise_tickets[team].append(ticket)

    return team_wise_tickets

In [76]:
def extract_tickets_into_list(team_wise_tickets):
    tickets_list = []
    for team in team_wise_tickets:
        for ticket in team_wise_tickets[team]:
            tickets_list.append(ticket)
    return tickets_list

In [77]:
team_wise_tickets = aggregate_reviews_teamwise()
tickets_list = extract_tickets_into_list(team_wise_tickets)

In [79]:
len(tickets_list)

13

In [78]:
# same in tickets.json
with open("datas/extracted_tickets.json","w") as f:
    json.dump(tickets_list,f,indent=4)

### JSONify 